# Projeto Final - Análise de Gols em uma Partida de Futebol
Autor: Paulo Victor Lima |   Orientador : Sergio Lima Netto |    Universidade Federal do Rio de Janeiro - Escola Politécnica - Departamente de Engenharia Eletrônica e de Computação

## Importação das Bibliotecas

In [1]:
#Instalação das bibliotecas

# !pip install pandas
# !pip install numpy
# !pip install seaborn
# !pip install matplotlib.pyplot
# !pip install plotly.express
# !pip install glob

In [2]:
#Importação das bibliotecas

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import glob
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import seaborn as sns
from numpy import loadtxt


## Importação Base de Dados

### Set Config

Classe set_config()

In [3]:
class set_config():
    def __init__(self, match_generic_path, highlights_file_name):

        #Carrega as features a serem consideradas
        features_types = []
        features_types = loadtxt("..\config\Features_Types.dat", comments="#", delimiter='\n', dtype=str, ndmin=1)
        self.features_types = features_types

        ## Caminho padrão das partidas
        self.match_generic_path = match_generic_path

        # Nome Padrão arquivo "Highlights"
        self.highlights_file_name = highlights_file_name

set_config() para treinamentos

In [4]:
def set_config_treinamento():
    
        match_generic_path = '..\\database\\Treinamento\\**\\'

        highlights_file_name = 'highlights.csv'
        
        return set_config(match_generic_path, highlights_file_name)

set_config() para testes

In [5]:
def set_config_teste():
    
        match_generic_path = '..\\database\\Teste\\**\\'

        highlights_file_name = 'highlights.csv'
        
        return set_config(match_generic_path, highlights_file_name)

### Leitura Base de Dados

Definição das Funções de leitura da base de dados

In [6]:
def import_moments_classifictaion(match, len, highlights_file_name):
    
    #Caminho para os arquivos de features
    match_by_feature_path = match + highlights_file_name

    #Data frame com todos os intervalos de highligths de uma partida
    highlights_file =  pd.DataFrame()

    #Data frame com os intervalos de highligths de uma partida
    highlights_of_match = pd.DataFrame(list(range(1,len)), columns=['classification'], index=list(range(1,len)))

    #nome das colunas do frame de highlights
    colnames=['first_frame','last_frame', 'classification'] 

    #Data Frame com todos os highlights de uma partida
    highlights_file = pd.read_csv(match_by_feature_path, names=colnames, index_col=None, sep =',', header=None)

    highlights_of_match['classification'] = np.nan
    
    for index,row in highlights_file.iterrows():        
        first_frame = int(row['first_frame']) if int(row['first_frame']) != ' ' else 0
        last_frame = int(row['last_frame']) if row['last_frame'] != ' ' else int(row['first_frame'])
        classification = row['classification'] if row['classification'] == ' Gol' else 'Não Gol'
        highlights_of_match.loc[first_frame:last_frame, 'classification'] = classification
        
    return highlights_of_match['classification']


In [7]:
def import_database (set_config):

    from numpy import loadtxt

    #Carrega as features a serem consideradas
    features_types = []
    features_types = set_config.features_types

    ## Caminho padrão das partidas
    match_generic_path = set_config.match_generic_path

    #Lista de partidas existentes na base
    all_matches = glob.glob(match_generic_path, recursive = False)

    ## Data Frame com todos os momementos de todas as partidas
    all_moments = pd.DataFrame()
    ## Array de Data Frames com todos os Data Frames das partidas
    all_moments_array = []

    for match in all_matches:
        print(match)

        #Data frame com todos os momentos de uma partida
        moments_of_match=  pd.DataFrame()

        for feature in features_types:

            #Caminho para os arquivos de features
            match_by_feature_path = match + feature

            #Data Frame com todos os momentos de uma partida de uma feature especifica
            moments_of_match[feature] = pd.read_csv(match_by_feature_path, index_col=None, sep ='\n', thousands=r".", header=None)
        
        #Preenche a coluna 'Match' com a partida
        moments_of_match['Match'] = [match_by_feature_path.split('\\')[-2] for _ in range(len(moments_of_match[feature]))]

            
        moments_of_match['Classification'] = import_moments_classifictaion(match, len(moments_of_match[feature]), set_config.highlights_file_name)

        moments_of_match.dropna(axis=0, how='any',inplace=True)

        #Adiciona ao array de Data Frames o Data Frame gerado para a partida
        all_moments_array.append(moments_of_match)

    all_moments = pd.concat(all_moments_array,  sort=False, axis=0, ignore_index=True)


    all_moments.to_csv( set_config.match_generic_path.split('\\')[-3] + "moments.csv", index=False, sep=';')

    return all_moments

    

## Divisão Previsores e Classe

Previsores

In [8]:
def get_predictors(all_moments):
    return all_moments.iloc[:, :-2].values

    

Classes

In [9]:
def get_classes(all_moments):
    return all_moments.iloc[:, -1:].values

## Função Principal

In [ ]:
########### TREINAMENTO ##########

# Configurações Gerais de Treinamento
set_config_treinamento = set_config_treinamento()

# Importa a base de dados
all_moments_treinamento = pd.DataFrame()
all_moments_treinamento = import_database(set_config_treinamento)

# Separa Preditores e Classificadores
predictors_treinamento = get_predictors(all_moments_treinamento)
classes_treinamento = get_classes(all_moments_treinamento)

##Cria o Random Forest e realiza o treinamento
random_forest = RandomForestClassifier(n_estimators=10,criterion='entropy',random_state=0)
random_forest.fit(predictors_treinamento, classes_treinamento)


############ TESTE ##########
# Configurações Gerais de Teste
set_config_teste = set_config_teste()

# Importa a base de dados
all_moments_teste = pd.DataFrame()
all_moments_teste = import_database(set_config_teste)

# Separa Preditores e Classificadores
predictors_teste = get_predictors(all_moments_teste)
classes_teste = get_classes(all_moments_teste)

predictions = random_forest.predict(predictors_teste)

############ Verificacao ##########
accuracy_score(classes_teste, predictions)



In [14]:
##Cria o Random Forest e realiza o treinamento
random_forest = RandomForestClassifier(n_estimators=1000,criterion='entropy',random_state=0)
random_forest.fit(predictors_treinamento, classes_treinamento)

predictions = random_forest.predict(predictors_teste)

############ Verificacao ##########
accuracy_score(classes_teste, predictions)

C:\Users\pv019\AppData\Local\Temp/ipykernel_19588/3501657085.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random_forest.fit(predictors_treinamento, classes_treinamento)


0.922226821192053

## Análises Auxiliares

In [11]:
all_moments = import_database()
sns.countplot(y="Classification", hue="Match", data = all_moments)

TypeError: import_database() missing 1 required positional argument: 'set_config'

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.countplot(y="Classification", data = all_moments)